In [1]:
import torch
import torchvision.transforms.functional as F

from PIL import Image

from glob import glob

import os

import pickle

/home/kdhsimplepro/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../data/celeba_male_number.pickle", "rb") as f:
    male_number = pickle.load(f)

print(len(male_number), male_number[:3])

7443 [8854, 28687, 5265]


In [3]:
from tqdm import tqdm

male_number_to_img = {}

# img_paths = glob("../data/CelebAMask-HQ/CelebA-HQ-img/*.jpg")

for number in tqdm(male_number):

    img_path = os.path.join("../data/CelebAMask-HQ/CelebA-HQ-img/", f"{number}.jpg")
    male_number_to_img["/".join(img_path.split("/")[2:])] = F.to_tensor(Image.open(img_path).convert("RGB").resize((64, 64)))

100%|██████████| 7443/7443 [00:50<00:00, 147.35it/s]


In [4]:
male_paths = glob("../data/male/*.jpg")

criterion = torch.nn.MSELoss()

male_path_to_celeba_path = {}

for path in tqdm(male_paths):
    img = F.to_tensor(Image.open(path).convert("RGB").resize((64, 64)))
    min_mse = 1e16
    celeba_num = 0

    for key, value in male_number_to_img.items():
        
        if key in male_path_to_celeba_path.values():
            continue

        mse = criterion(img, value)

        if mse < min_mse:
            min_mse = mse
            celeba_num = key

    male_path_to_celeba_path["/".join(path.split("/")[2:])] = os.path.join("CelebAMask-HQ", "CelebA-HQ-img", f"{celeba_num}.jpg")

100%|██████████| 7443/7443 [28:26<00:00,  4.36it/s]


In [7]:
print(len(male_path_to_celeba_path), list(male_path_to_celeba_path.items())[:10])

7443 [('male/1716.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/26865.jpg.jpg'), ('male/7136.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/25891.jpg.jpg'), ('male/3811.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/6727.jpg.jpg'), ('male/5649.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/22259.jpg.jpg'), ('male/3815.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/21187.jpg.jpg'), ('male/4284.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/25963.jpg.jpg'), ('male/5265.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/9482.jpg.jpg'), ('male/6449.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/7310.jpg.jpg'), ('male/7324.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/12649.jpg.jpg'), ('male/4484.jpg', 'CelebAMask-HQ/CelebA-HQ-img/CelebAMask-HQ/CelebA-HQ-img/15544.jpg.jpg')]


In [ ]:
with open("./male_path_to_celeba_path.pickle", "wb") as f:
    pickle.dump(male_path_to_celeba_path, f)